In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from datasets import Dataset

from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

In [35]:
z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)

In [36]:

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [37]:
encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surprised": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "n

In [39]:
batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [40]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [41]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



In [42]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9133
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5710


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.255900,0.218342,0.473684,0.658680,0.322581
2,0.209000,0.203539,0.651685,0.755907,0.505376
3,0.145200,0.244627,0.687179,0.789812,0.537634
4,0.108700,0.254192,0.673575,0.780637,0.537634
5,0.088700,0.266485,0.696970,0.798064,0.537634


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi

TrainOutput(global_step=5710, training_loss=0.16345449370803433, metrics={'train_runtime': 797.427, 'train_samples_per_second': 57.265, 'train_steps_per_second': 7.161, 'total_flos': 3003876432672000.0, 'train_loss': 0.16345449370803433, 'epoch': 5.0})

In [43]:
trainer.save_model('fine_tune_bert_1')

Saving model checkpoint to fine_tune_bert_1
Configuration saved in fine_tune_bert_1/config.json
Model weights saved in fine_tune_bert_1/pytorch_model.bin
tokenizer config file saved in fine_tune_bert_1/tokenizer_config.json
Special tokens file saved in fine_tune_bert_1/special_tokens_map.json


In [ ]:
model = "microsoft/deberta-base"
pth = 'fine_tune_bert_2'


tokenizer = AutoTokenizer.from_pretrained(model)
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)


'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model, 
                problem_type="multi_label_classification", 
                num_labels=len(labels),
                id2label=id2label,
                label2id=label2id)


batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model(pth)

loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "c2p",
    "p2c"
  ],
  "position_biased_input": false,
  "relative_attention": true,
  "transformers_version": "4.21.1",
  "type_vocab_size": 0,
  "vocab_size": 50265
}

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surprised": 3
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.253800,0.205974,0.577181,0.702830,0.387097
2,0.210200,0.164952,0.697143,0.780396,0.548387
3,0.164100,0.182854,0.776596,0.836083,0.655914
4,0.120600,0.184433,0.753927,0.827696,0.645161
5,0.089900,0.199377,0.752577,0.830578,0.655914


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi

In [27]:
model = "bert-large-uncased"
pth = 'fine_tune_bert_3'


tokenizer = AutoTokenizer.from_pretrained(model)
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)


'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model, 
                problem_type="multi_label_classification", 
                num_labels=len(labels),
                id2label=id2label,
                label2id=label2id)


batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model(pth)

https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmptbptu9o9


storing https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-large-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpt_2uh1ol


storing https://huggingface.co/bert-large-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
creating metadata file for /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
loading configuration file https://huggingface.co/bert-large-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
 

storing https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpg4dzret8


storing https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://h

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-large-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surprised": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
 

storing https://huggingface.co/bert-large-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/1d959166dd7e047e57ea1b2d9b7b9669938a7e90c5e37a03961ad9f15eaea17f.fea64cd906e3766b04c92397f9ad3ff45271749cbe49829a079dd84e34c1697d
creating metadata file for /root/.cache/huggingface/transformers/1d959166dd7e047e57ea1b2d9b7b9669938a7e90c5e37a03961ad9f15eaea17f.fea64cd906e3766b04c92397f9ad3ff45271749cbe49829a079dd84e34c1697d
loading weights file https://huggingface.co/bert-large-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1d959166dd7e047e57ea1b2d9b7b9669938a7e90c5e37a03961ad9f15eaea17f.fea64cd906e3766b04c92397f9ad3ff45271749cbe49829a079dd84e34c1697d
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.tr

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.260900,0.249367,0.619883,0.742011,0.537634
2,0.222100,0.212300,0.670588,0.765812,0.559140
3,0.168600,0.199996,0.666667,0.764898,0.569892
4,0.120000,0.247443,0.728261,0.810232,0.666667
5,0.089200,0.268326,0.729167,0.820085,0.677419


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi

In [29]:
model = "deepset/roberta-base-squad2"
pth = 'fine_tune_bert_4'


tokenizer = AutoTokenizer.from_pretrained(model)
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)


'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model, 
                problem_type="multi_label_classification", 
                num_labels=len(labels),
                id2label=id2label,
                label2id=label2id)


batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model(pth)

https://huggingface.co/deepset/roberta-base-squad2/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpx_khkymr


storing https://huggingface.co/deepset/roberta-base-squad2/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513
creating metadata file for /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513
https://huggingface.co/deepset/roberta-base-squad2/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp9398zh15


storing https://huggingface.co/deepset/roberta-base-squad2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673
creating metadata file for /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673
loading configuration file https://huggingface.co/deepset/roberta-base-squad2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gra

storing https://huggingface.co/deepset/roberta-base-squad2/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
creating metadata file for /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
https://huggingface.co/deepset/roberta-base-squad2/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpouwuw7gm


storing https://huggingface.co/deepset/roberta-base-squad2/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/deepset/roberta-base-squad2/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp856heh5_


...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/deepset/roberta-base-squad2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surprised": 3
  },
  "language"

storing https://huggingface.co/deepset/roberta-base-squad2/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e
creating metadata file for /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e
loading weights file https://huggingface.co/deepset/roberta-base-squad2/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e
Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaForSequenceClassification: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaForSequenceClassification

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.264300,0.248644,0.612500,0.729178,0.505376
2,0.228500,0.204102,0.615385,0.727113,0.494624
3,0.193100,0.209608,0.638037,0.743601,0.537634
4,0.156400,0.240562,0.716578,0.807490,0.655914
5,0.135800,0.244173,0.725389,0.819171,0.666667


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi

In [30]:
model = "UBC-NLP/MARBERT"
pth = 'fine_tune_bert_5'


tokenizer = AutoTokenizer.from_pretrained(model)
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)


'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model, 
                problem_type="multi_label_classification", 
                num_labels=len(labels),
                id2label=id2label,
                label2id=label2id)


batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model(pth)

https://huggingface.co/UBC-NLP/MARBERT/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7uwmepxv


storing https://huggingface.co/UBC-NLP/MARBERT/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c3fd5ae192b6ecfac21e1ee10a127a4be3a8f11f15ba705dfbe28df91892907d.add0f663209440c3da3f309e22b445abb9a90c8572bbe49423f50103baf5ab17
creating metadata file for /root/.cache/huggingface/transformers/c3fd5ae192b6ecfac21e1ee10a127a4be3a8f11f15ba705dfbe28df91892907d.add0f663209440c3da3f309e22b445abb9a90c8572bbe49423f50103baf5ab17
https://huggingface.co/UBC-NLP/MARBERT/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmptrwl2k4m


storing https://huggingface.co/UBC-NLP/MARBERT/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/149fd0f2228ba9e18736429c365b97eb4a2af1c198b1e068b99815d78d3d9e25.897c52772d161fa08b3d031e0e548347e8ae1f81959cd6e58ccf096423ca5a11
creating metadata file for /root/.cache/huggingface/transformers/149fd0f2228ba9e18736429c365b97eb4a2af1c198b1e068b99815d78d3d9e25.897c52772d161fa08b3d031e0e548347e8ae1f81959cd6e58ccf096423ca5a11
loading configuration file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/149fd0f2228ba9e18736429c365b97eb4a2af1c198b1e068b99815d78d3d9e25.897c52772d161fa08b3d031e0e548347e8ae1f81959cd6e58ccf096423ca5a11
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dr

storing https://huggingface.co/UBC-NLP/MARBERT/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/c1d67f93c84050eb506184e5625d53c4946bd98af9c62ad1ee92d4db1559105a.9985cd6ca030442c4f68221160381b229fee63902f75a8f43e14e78007536585
creating metadata file for /root/.cache/huggingface/transformers/c1d67f93c84050eb506184e5625d53c4946bd98af9c62ad1ee92d4db1559105a.9985cd6ca030442c4f68221160381b229fee63902f75a8f43e14e78007536585
https://huggingface.co/UBC-NLP/MARBERT/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpkxt2v9ta


storing https://huggingface.co/UBC-NLP/MARBERT/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6826635eba590bd92524e8730921a4029f540c03dbc23e3630c38d49b85b2d0d.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/6826635eba590bd92524e8730921a4029f540c03dbc23e3630c38d49b85b2d0d.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/c1d67f93c84050eb506184e5625d53c4946bd98af9c62ad1ee92d4db1559105a.9985cd6ca030442c4f68221160381b229fee63902f75a8f43e14e78007536585
loading file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/special_tokens_map

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/149fd0f2228ba9e18736429c365b97eb4a2af1c198b1e068b99815d78d3d9e25.897c52772d161fa08b3d031e0e548347e8ae1f81959cd6e58ccf096423ca5a11
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surprised": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "m

storing https://huggingface.co/UBC-NLP/MARBERT/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/d2c40ae07966159d912382ac272d518da4833484bbb486d4af85adb250ff3702.3f1282b471bf663bd7cc69cf2161fb3a56e95efa917842b75be8301a8c58ce1c
creating metadata file for /root/.cache/huggingface/transformers/d2c40ae07966159d912382ac272d518da4833484bbb486d4af85adb250ff3702.3f1282b471bf663bd7cc69cf2161fb3a56e95efa917842b75be8301a8c58ce1c
loading weights file https://huggingface.co/UBC-NLP/MARBERT/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/d2c40ae07966159d912382ac272d518da4833484bbb486d4af85adb250ff3702.3f1282b471bf663bd7cc69cf2161fb3a56e95efa917842b75be8301a8c58ce1c
Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.d

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.276800,0.218406,0.549020,0.693526,0.408602
2,0.242500,0.195346,0.598726,0.718028,0.430108
3,0.206300,0.193862,0.629630,0.735979,0.473118
4,0.169900,0.215639,0.685714,0.774762,0.548387
5,0.143600,0.221909,0.706522,0.794677,0.580645


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi

In [46]:
model = "xlm-roberta-large-finetuned-conll03-english"
pth = 'fine_tune_bert_6'


tokenizer = AutoTokenizer.from_pretrained(model)
df = pd.read_csv('cleaned_non-vectorized_data_2.csv')

z1 = tokenizer(df.loc[0, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']
list_out = []
for i in range(1, len(df)):
    try:
        z2 = tokenizer(df.loc[i, 'Text'], padding="max_length", truncation=True, max_length=128)['input_ids']

        z1 = np.vstack((z1, z2))
    except:
        list_out.append(i)
#         print(i)
        continue

text = torch.tensor(z1)
df2 = df.copy()

df2.drop(df2.index[list_out], inplace=True)
df2 = df2.reset_index()

labels = list(df.columns[2:])
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

# arr_label = torch.tensor(df2.loc[:, labels].values)

# train = data_utils.TensorDataset(text, arr_label)
# eval_data = data_utils.TensorDataset(text, arr_label)
# train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)
# eval_loader = data_utils.DataLoader(eval_data, batch_size=100, shuffle=True)

df2 = df2.drop(columns=['tokens'])

train, test = train_test_split(df2, test_size=0.01)
train_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_dict(test)


'''
rerference: https://discuss.huggingface.co/t/sending-a-dataset-or-datasetdict-to-a-gpu/17208/4

'''

def preprocess_data(examples):
    # take a batch of texts
    text = examples["Text"]
    
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
#     print(examples)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

encoded_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
endoded_valid = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model, 
                problem_type="multi_label_classification", 
                num_labels=len(labels),
                id2label=id2label,
                label2id=label2id)


batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-multilabel-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.015,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.75):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=endoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


trainer.save_model(pth)

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpp1fa5p8d


storing https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/cd7e3332fb70fd94304b0fcad297caff7b8d9c97c5f76b5a4ac6bbcc14379fe1.ed0120fc465ef220e4bd136ae002fa78741a9545246ccb78502333b8dba60ee3
creating metadata file for /root/.cache/huggingface/transformers/cd7e3332fb70fd94304b0fcad297caff7b8d9c97c5f76b5a4ac6bbcc14379fe1.ed0120fc465ef220e4bd136ae002fa78741a9545246ccb78502333b8dba60ee3
loading configuration file https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cd7e3332fb70fd94304b0fcad297caff7b8d9c97c5f76b5a4ac6bbcc14379fe1.ed0120fc465ef220e4bd136ae002fa78741a9545246ccb78502333b8dba60ee3
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large-finetuned-conll03-english",
  "_num_labels": 8,
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  

storing https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/sentencepiece.bpe.model in cache at /root/.cache/huggingface/transformers/c5604a1be93150d0d1b8dfa45818ac04c1261ea33b5aa73e9f62b07171cafd93.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
creating metadata file for /root/.cache/huggingface/transformers/c5604a1be93150d0d1b8dfa45818ac04c1261ea33b5aa73e9f62b07171cafd93.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp83d9wfpf


storing https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/b32bffe3a0720df8952ddee5d7b1f9dd5c71ac8f0e0a69ba69432d8ab0be410c.1b58f47e7fc4532adbdc01d216b9bb2fb0657db965b423d3c9c974934c7c50e3
creating metadata file for /root/.cache/huggingface/transformers/b32bffe3a0720df8952ddee5d7b1f9dd5c71ac8f0e0a69ba69432d8ab0be410c.1b58f47e7fc4532adbdc01d216b9bb2fb0657db965b423d3c9c974934c7c50e3
loading file https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/sentencepiece.bpe.model from cache at /root/.cache/huggingface/transformers/c5604a1be93150d0d1b8dfa45818ac04c1261ea33b5aa73e9f62b07171cafd93.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/b32bffe3a0720df8952ddee5d7b1f9dd5c71ac8f0e0a69ba69432d8ab0be410

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cd7e3332fb70fd94304b0fcad297caff7b8d9c97c5f76b5a4ac6bbcc14379fe1.ed0120fc465ef220e4bd136ae002fa78741a9545246ccb78502333b8dba60ee3
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large-finetuned-conll03-english",
  "_num_labels": 8,
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "disgust",
    "1": "joy",
    "2": "anger",
    "3": "surprised",
    "4": "sad",
    "5": "fear",
    "6": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "anger": 2,
    "disgust": 0,
    "fear": 5,
    "joy": 1,
    "neutral": 6,
    "sad": 4,
    "surpr

storing https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/2b449384f083d8866edacbf92e4011e3fcf3026d45f2b9a4cb8650fec7a525c7.09ff0f236572ba82656162f6d6e7ec75e1af5babbf6a088165855208ad7a2c6d
creating metadata file for /root/.cache/huggingface/transformers/2b449384f083d8866edacbf92e4011e3fcf3026d45f2b9a4cb8650fec7a525c7.09ff0f236572ba82656162f6d6e7ec75e1af5babbf6a088165855208ad7a2c6d
loading weights file https://huggingface.co/xlm-roberta-large-finetuned-conll03-english/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/2b449384f083d8866edacbf92e4011e3fcf3026d45f2b9a4cb8650fec7a525c7.09ff0f236572ba82656162f6d6e7ec75e1af5babbf6a088165855208ad7a2c6d
Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForSequenceClassification: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bi

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.287400,0.227134,0.526316,0.684995,0.408602
2,0.259100,0.215847,0.621118,0.733986,0.505376
3,0.232000,0.224743,0.594937,0.719563,0.473118
4,0.199200,0.239539,0.666667,0.767877,0.559140
5,0.174600,0.250727,0.674286,0.772685,0.569892


***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-1142
Configuration saved in bert-finetuned-multilabel-english/checkpoint-1142/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-1142/tokenizer_config.json
Special tokens file saved in bert-finetuned-multilabel-english/checkpoint-1142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 93
  Batch size = 8
Saving model checkpoint to bert-finetuned-multilabel-english/checkpoint-2284
Configuration saved in bert-finetuned-multilabel-english/checkpoint-2284/config.json
Model weights saved in bert-finetuned-multilabel-english/checkpoint-2284/pytorch_model.bin
tokenizer config file saved in bert-finetuned-multilabel-english/checkpoint-2284/tokenizer_config.json
Special tokens file saved in bert-finetuned-multi